# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# original_df = pd.read_csv(file)
# original_df.head()
types_df = pd.read_csv("../output_data/HW.csv")
types_df.head()

,City,Max Temp,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
0,Bilma,288.27,34,12.9164,18.6853,0,NE,6.32,1612405246
1,Yarmouth,274.82,75,-70.2286,41.7057,90,US,1.54,1612405246
2,Busselton,292.04,53,115.3333,-33.6500,0,AU,1.34,1612405248
3,Cayenne,300.15,83,-52.3333,4.9333,75,GF,6.17,1612405249
4,Punta Arenas,288.15,77,-70.9167,-53.1500,0,CL,6.17,1612405249


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
locations = types_df[["Latitude", "Longitude"]]
Humidity= types_df["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
# This is Jason file, Temp is wrong from API
# 'coord': {'lon': 45.3438, 'lat': 2.0371},
#  'weather': [{'id': 802,
#    'main': 'Clouds',
#    'description': 'scattered clouds',
#    'icon': '03d'}],
#  'base': 'stations',
#  'main': {'temp': 302.15,
#   'feels_like': 299.14,
#   'temp_min': 302.15,
#   'temp_max': 302.15,
#   'pressure': 1013,
#   'humidity': 54},
#  'visibility': 10000,
#  'wind': {'speed': 8.75, 'deg': 60},
#  'clouds': {'all': 40},
#  'dt': 1612424454,
#  'sys': {'type': 1,
#   'id': 2497,
#   'country': 'SO',
#   'sunrise': 1612408286,
#   'sunset': 1612451618},
#  'timezone': 10800,
#  'id': 53654,
#  'name': 'Mogadishu',
#  'cod': 200}"

types_df.head()
Max=types_df.query('`Max Temp` < 300')
Min=Max.query('`Max Temp` > 100')
#This is a reptile planet aparently
Wind=Min.query('Windspeed > 10')
hotel_df=Wind.query('Cloudiness>15')
hotel_df

,City,Max Temp,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
51,Kahului,293.15,77,-156.4700,20.8947,40,US,12.35,1612405267
91,Ribeira Grande,286.35,65,-28.7000,38.5167,79,PT,13.10,1612405285
95,Geraldton,297.15,50,114.6000,-28.7667,90,AU,12.35,1612405022
123,Okhotsk,249.65,82,143.3000,59.3833,100,RU,10.63,1612405303
135,Pierre,272.15,80,-100.3510,44.3683,90,US,10.29,1612405310
144,Vestmannaeyjar,276.19,79,-20.2734,63.4427,100,IS,13.93,1612405313
150,Lagoa,285.64,64,-27.9833,39.0500,99,PT,13.64,1612405319
186,Bathsheba,299.15,69,-59.5167,13.2167,40,BB,11.83,1612405336
224,Mandan,265.15,92,-100.8896,46.8267,90,US,10.29,1612405351
232,Sørland,274.97,80,12.6934,67.6670,70,NO,11.40,1612405354


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [79]:
hotel_df["Hotel Name"] = ''


<ipython-input-79-e27eeba5f9b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ''


In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json
params = {
    "location": "39.952583,-75.16522", 
    "rankby": "distance",
    "type": "City",
    "key": gkey,
for index, row in hotel_df.iterrows():
      name_city = row['City']
    
     response = requests.get(base_url, params=params).json()
    
     results = response['results']

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [82]:
marker_locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
fig.add_layer(markers)
fig

NameError: name 'City' is not defined